In [ ]:
# -------------------------------
# Imports
# -------------------------------
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from twilio.rest import Client

# -------------------------------
# Parameters
# -------------------------------
IMG_HEIGHT, IMG_WIDTH = 128, 128

# Twilio credentials (Yuthika's)
TWILIO_ACCOUNT_SID = "YOUR_ACCOUNT_SID"
TWILIO_AUTH_TOKEN = "YOUR_AUTH_TOKEN"
TWILIO_PHONE_NUMBER = "+12162847314"
RECIPIENT_PHONE_NUMBER = "+919962012439"

# Initialize Twilio client
twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

# File to store predicted expiry dates
EXPIRY_RECORD_FILE = r"C:\Users\Yuthika\FOOD EXPIRY DETECTION\expiry_dates.csv"

# -------------------------------
# Load trained model
# -------------------------------
MODEL_PATH = r"C:\Users\Yuthika\FOOD EXPIRY DETECTION\fruit_expiry_regression_model.h5"
model = load_model(MODEL_PATH)

# -------------------------------
# Preprocess image
# -------------------------------
def preprocess_image(path):
    img = load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

# -------------------------------
# Send SMS
# -------------------------------
def send_sms_notification(message_text, recipient_number):
    try:
        message = twilio_client.messages.create(
            body=message_text,
            from_=TWILIO_PHONE_NUMBER,
            to=recipient_number
        )
        print(f"✅ SMS sent successfully! SID: {message.sid}")
    except Exception as e:
        print(f"❌ SMS sending failed: {e}")

# -------------------------------
# Predict, store, notify
# -------------------------------
def predict_single_image(image_path):
    img_array = preprocess_image(image_path)
    raw_pred = model.predict(img_array)[0][0]

    print(f"🔍 Raw model output (before adjustment): {raw_pred:.2f}")

    # --- Invert scale if smaller numbers mean fresher fruit ---
    predicted_days = 10 - float(np.clip(raw_pred, 0, 10))

    # Convert to expiry date
    expiry_date = datetime.date.today() + datetime.timedelta(days=predicted_days)
    fruit_name = os.path.basename(image_path)

    # Save to CSV
    if os.path.exists(EXPIRY_RECORD_FILE):
        df = pd.read_csv(EXPIRY_RECORD_FILE, parse_dates=['expiry_date'])
    else:
        df = pd.DataFrame(columns=['fruit_name', 'expiry_date'])

    df = pd.concat([
        df,
        pd.DataFrame({'fruit_name': [fruit_name], 'expiry_date': [expiry_date]})
    ], ignore_index=True)
    df.to_csv(EXPIRY_RECORD_FILE, index=False)

    # -------------------------------
    # Logical thresholds
    # -------------------------------
    today = datetime.date.today()
    days_left = (expiry_date - today).days

    print("\n------------------------------")
    print(f"🧺 Fruit: {fruit_name}")
    print(f"📅 Predicted Expiry Date: {expiry_date.strftime('%d/%m/%y')}")
    print(f"Predicted days left: {predicted_days:.2f}")

    if predicted_days >= 7:
        status = "✅ The food is fresh and good to use."
        message_text = f"📢 Info: {fruit_name} is fresh. Expected to expire on {expiry_date.strftime('%d/%m/%y')}."
    elif predicted_days >= 3:
        status = "⚠️ The food is nearing expiry."
        message_text = f"⚠️ Reminder: {fruit_name} will expire soon (in about {days_left} days)."
    else:
        status = "❌ The food is expired or spoiled."
        message_text = f"❌ {fruit_name} has expired. Please discard it safely."

    print(status)
    print("------------------------------\n")

    send_sms_notification(message_text, RECIPIENT_PHONE_NUMBER)

# -------------------------------
# Example usage
# -------------------------------
if __name__ == "__main__":
    # Update your image path here
    image_path = r"C:\Users\Yuthika\FOOD EXPIRY DETECTION\orange.png"
    predict_single_image(image_path)

ModuleNotFoundError: No module named 'tensorflow'